In [76]:
!pip install keras

In [77]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.preprocessing import OneHotEncoder

In [78]:
data=pd.read_csv("newcleaneddata.csv")
print(data.columns.tolist())

['Product_Category', 'Product_Price', 'Order_Quantity', 'Return_Reason', 'Return_Status', 'User_Age', 'User_Gender', 'User_Location', 'Payment_Method', 'Shipping_Method']


In [79]:
data = data.drop('Product_Price', axis=1)
data = data.drop('Order_Quantity', axis=1)
data = data.drop('User_Location', axis=1)


In [80]:
##encode categorical value
from sklearn.preprocessing import LabelEncoder

# Encode User_Gender
label_encoder_gender = LabelEncoder()
data['User_Gender'] = label_encoder_gender.fit_transform(data['User_Gender'])

# Encode Return_Status (Target)
label_encoder_status = LabelEncoder()
data['Return_Status'] = label_encoder_status.fit_transform(data['Return_Status'])


In [81]:
import pickle

with open('gender_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder_gender, f)

with open('status_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder_status, f)


In [82]:
from sklearn.preprocessing import OneHotEncoder

# Initialize the OneHotEncoder
payment_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
payment_encoded = payment_encoder.fit_transform(data[['Payment_Method']])
payment_encoded_df = pd.DataFrame(payment_encoded, columns=payment_encoder.get_feature_names_out(['Payment_Method']))
with open('payment_encoder.pkl', 'wb') as f:
    pickle.dump(payment_encoder, f)


In [83]:
data = pd.concat([data.drop('Payment_Method', axis=1), payment_encoded_df], axis=1)

In [84]:
return_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Fit-transform 'Return_Reason'
return_encoded = return_encoder.fit_transform(data[['Return_Reason']])

# Create a DataFrame with appropriate column names
return_encoded_df = pd.DataFrame(return_encoded, columns=return_encoder.get_feature_names_out(['Return_Reason']))

# Reset index for safe merging later (if needed)
return_encoded_df.reset_index(drop=True, inplace=True)

# Preview the result
print(return_encoded_df.head())

   Return_Reason_Changed mind  Return_Reason_Defective  \
0                         1.0                      0.0   
1                         0.0                      0.0   
2                         0.0                      0.0   
3                         0.0                      0.0   
4                         0.0                      0.0   

   Return_Reason_Not as described  Return_Reason_Wrong item  Return_Reason_nan  
0                             0.0                       0.0                0.0  
1                             0.0                       1.0                0.0  
2                             0.0                       0.0                1.0  
3                             0.0                       0.0                1.0  
4                             0.0                       0.0                1.0  


In [85]:
data = pd.concat([data.drop('Return_Reason', axis=1),return_encoded_df ], axis=1)

In [86]:
shipping_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Fit and transform the 'Shipping_Method' column
shipping_encoded = shipping_encoder.fit_transform(data[['Shipping_Method']])

# Create DataFrame with proper column names
shipping_encoded_df = pd.DataFrame(
    shipping_encoded,
    columns=shipping_encoder.get_feature_names_out(['Shipping_Method'])
)

# Reset index (optional, for future concatenation)
shipping_encoded_df.reset_index(drop=True, inplace=True)

# Display result
print(shipping_encoded_df.head())

   Shipping_Method_Express  Shipping_Method_Next-Day  Shipping_Method_Standard
0                      0.0                       1.0                       0.0
1                      1.0                       0.0                       0.0
2                      0.0                       1.0                       0.0
3                      0.0                       1.0                       0.0
4                      0.0                       0.0                       1.0


In [87]:
data = pd.concat([data.drop('Shipping_Method', axis=1),shipping_encoded_df ], axis=1)

In [88]:
from sklearn.preprocessing import LabelEncoder

# Initialize the encoder
product_label_encoder = LabelEncoder()

# Apply Label Encoding to 'Product_Category'
data['Product_Category'] = product_label_encoder.fit_transform(data['Product_Category'])

# (Optional) Save the encoder for future predictions
import pickle
with open('product_label_encoder.pkl', 'wb') as f:
    pickle.dump(product_label_encoder, f)

# Preview
print(data['Product_Category'].head())




0    1
1    0
2    4
3    4
4    0
Name: Product_Category, dtype: int64


In [89]:
import pickle

# Save LabelEncoder for Product_Category (if used)
with open('product_label_encoder.pkl', 'wb') as file:
    pickle.dump(product_label_encoder, file)

# Save OneHotEncoders
with open('shipping_encoder.pkl', 'wb') as file:
    pickle.dump(shipping_encoder, file)

with open('return_reason_encoder.pkl', 'wb') as file:
    pickle.dump(return_encoder, file)



In [90]:
print(data.columns)


Index(['Product_Category', 'Return_Status', 'User_Age', 'User_Gender',
       'Payment_Method_Credit Card', 'Payment_Method_Debit Card',
       'Payment_Method_Gift Card', 'Payment_Method_PayPal',
       'Return_Reason_Changed mind', 'Return_Reason_Defective',
       'Return_Reason_Not as described', 'Return_Reason_Wrong item',
       'Return_Reason_nan', 'Shipping_Method_Express',
       'Shipping_Method_Next-Day', 'Shipping_Method_Standard'],
      dtype='object')


In [91]:
print(data.columns)


Index(['Product_Category', 'Return_Status', 'User_Age', 'User_Gender',
       'Payment_Method_Credit Card', 'Payment_Method_Debit Card',
       'Payment_Method_Gift Card', 'Payment_Method_PayPal',
       'Return_Reason_Changed mind', 'Return_Reason_Defective',
       'Return_Reason_Not as described', 'Return_Reason_Wrong item',
       'Return_Reason_nan', 'Shipping_Method_Express',
       'Shipping_Method_Next-Day', 'Shipping_Method_Standard'],
      dtype='object')


In [92]:
print(data.columns)

Index(['Product_Category', 'Return_Status', 'User_Age', 'User_Gender',
       'Payment_Method_Credit Card', 'Payment_Method_Debit Card',
       'Payment_Method_Gift Card', 'Payment_Method_PayPal',
       'Return_Reason_Changed mind', 'Return_Reason_Defective',
       'Return_Reason_Not as described', 'Return_Reason_Wrong item',
       'Return_Reason_nan', 'Shipping_Method_Express',
       'Shipping_Method_Next-Day', 'Shipping_Method_Standard'],
      dtype='object')


In [93]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pickle

# 1. Separate independent features (X) and the target (y)
X = data.drop('Return_Status', axis=1)
y = data['Return_Status']

# 2. Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 3. Apply scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 4. Save the scaler for future use
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

# ✅ Optional: Print shapes for verification
print("X_train shape:", X_train_scaled.shape)
print("X_test shape:", X_test_scaled.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


X_train shape: (8000, 15)
X_test shape: (2000, 15)
y_train shape: (8000,)
y_test shape: (2000,)


In [94]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Define the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # For binary classification
])

# Compile the model
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)


c:\Users\ANURAG RAWAT\anaconda3\envs\arnav\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [95]:
import datetime
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

# Set up the TensorBoard log directory with proper datetime format
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Create the TensorBoard callback
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Optional: Add early stopping to avoid overfitting
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


In [96]:
##Set up early stooping
early_stopping_callback=EarlyStopping(monitor='cal_loss',patience=5,restore_best_weights=True)

#train the model
history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=(tensor_flow_callback,early_stopping_callback)
)

In [97]:
model.save('return_ann_model_v1.h5')


In [98]:
!kill 25432


'kill' is not recognized as an internal or external command,
operable program or batch file.


In [99]:
%load_ext tensorboard
%tensorboard --logdir logs/fit --port 6006


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 6000), started 22:06:09 ago. (Use '!kill 6000' to kill it.)

In [100]:
%reload_ext tensorboard